In [122]:
## Eventually, I'd like to create a program which automatically pulls data from 
## the Japanese Ministry of Health, Labor and Welfare and imports it into a dashboard.

## Import packages
import pandas as pd
import numpy as np
import datetime as dt
import ssl

## Display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Data Sources, setting up SSL
ssl._create_default_https_context = ssl._create_unverified_context
dataCases = 'http://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv'
dataSevere = 'http://covid19.mhlw.go.jp/public/opendata/severe_cases_daily.csv'
dataDeaths = 'http://covid19.mhlw.go.jp/public/opendata/deaths_cumulative_daily.csv'

## First day of each COVID wave in Datetime format
dtWaves = [dt.datetime(2020, 7, 1),dt.datetime(2020, 11, 1),dt.datetime(2021, 3, 1),
           dt.datetime(2021, 7, 1),dt.datetime(2022, 1, 1),dt.datetime(2022, 6, 1),
           dt.datetime(2029, 6, 1)]

In [123]:
## Create and Join Data Frames
dfCases = pd.read_csv(dataCases)[['Date','ALL']]
dfCases.rename(columns = {'ALL':'newCases'}, inplace = True)

dfSevere = pd.read_csv(dataSevere)[['Date','ALL']]
dfSevere.rename(columns = {'ALL':'totalSevere'}, inplace = True)

dfDeaths = pd.read_csv(dataDeaths)[['Date','ALL']]
dfDeaths.rename(columns = {'ALL':'cumDeaths'}, inplace = True)

df = dfCases.join(dfSevere.set_index('Date'), on="Date", how='inner')
df2 = df.join(dfDeaths.set_index('Date'), on="Date", how='inner')

## Inputting each wave into the dataframe, using the dtWaves array above
def wave(row):
    for i in range(0,len(waves)):
        if dt.datetime.strptime(row['Date'], '%Y/%m/%d') >= dtWaves[i]:
            if dt.datetime.strptime(row['Date'], '%Y/%m/%d') < dtWaves[i + 1]:
                return "Wave " + str(i+2)
        else:
            return "Wave 1"

df2['wave'] = df2.apply(wave, axis=1)

## Adding column for new deaths per day
newDeaths = []
last = 613
for i in df2["cumDeaths"]:
    newDeaths.append(i - last)
    last = i

df2.insert(4, "newDeaths", newDeaths, True)

## Adding column for average number of weekly cases
weeklyAvgCases = []
avg = []

for i in df2["newCases"]:
    if len(avg) < 7:
        avg.append(i)
        weeklyAvgCases.append(None)
    else:
        del avg[0]
        avg.append(i)
        weeklyAvgCases.append(round(np.average(avg,axis=0),1))

df2.insert(2, "weeklyAvgCases", weeklyAvgCases, True)

## Adding column for average number of weekly deaths
weeklyAvgDeaths = []
avg = []

for i in df2["newDeaths"]:
    if len(avg) < 7:
        avg.append(i)
        weeklyAvgDeaths.append(None)
    else:
        del avg[0]
        avg.append(i)
        weeklyAvgDeaths.append(round(np.average(avg,axis=0),1))

df2.insert(6, "weeklyAvgDeaths", weeklyAvgDeaths, True)


## Adding three columns with a constant value containing all cases, all severe, and all deaths
totalCases = 0
totalSevere = 0
totalDeaths = 0
for i in df2["newCases"]:
    totalCases += i

for i in df2["totalSevere"]:
    totalSevere += i
    
for i in df2["newDeaths"]:
    totalDeaths += i
    
df2['allCases'] = pd.Series([totalCases for x in range(len(df2.index))])
df2['allSevere'] = pd.Series([totalSevere for x in range(len(df2.index))])
df2['allDeaths'] = pd.Series([totalDeaths for x in range(len(df2.index))])

In [124]:

print(df2)

           Date  newCases  weeklyAvgCases  totalSevere  cumDeaths  newDeaths  \
114    2020/5/9       108             NaN          267        613          0   
115   2020/5/10        66             NaN          249        621          8   
116   2020/5/11        58             NaN          243        643         22   
117   2020/5/12        87             NaN          259        668         25   
118   2020/5/13        55             NaN          245        687         19   
119   2020/5/14        99             NaN          237        710         23   
120   2020/5/15        55             NaN          232        725         15   
121   2020/5/16        56            68.0          230        744         19   
122   2020/5/17        29            62.7          228        749          5   
123   2020/5/18        30            58.7          213        763         14   
124   2020/5/19        32            50.9          210        771          8   
125   2020/5/20        37            48.